Used to create unique access codes for Lifted Electronic Message Pickup.  Also used to combine PDFs of multiple recipient messages and then rename them with "NetID_accessCode.pdf"

Also, used to format a CSV for MailMerge for sending these messages

In [165]:
import pandas as pd
import numpy as np
import re
import pypdf
import os
import secrets

In [184]:
def process_email(email_list):
    netIDs = email_list.str.split("@", n=1, expand=True)[0]
    lowercase_trimmed = netIDs.str.lower().str.strip()
    return lowercase_trimmed

In [185]:
sems = []
for file in os.listdir("submissions"):
    if "csv" in file:
        sems.append(file.split(".")[0])

sems.sort(key=lambda x: int(x.split("_")[0]), reverse=True)
sems

['10_sp_24_e',
 '9_sp_24_p',
 '8_sp_23_e',
 '7_sp_23_p',
 '6_sp_22_e',
 '5_sp_22_p',
 '4_sp_21_p',
 '3_sp_20_e',
 '2_sp_19_p',
 '1_sp_18_p']

In [186]:
db = pd.DataFrame(columns=["id"] + sems).set_index("id")
db

for sem in sems:
    print(sem)
    df = pd.read_csv(f'submissions/{sem}.csv')
    recipient_netids = process_email(df["Recipient's email"])
    val_counts_df = pd.DataFrame(recipient_netids.value_counts().rename(sem))
    db = pd.concat([db, val_counts_df])
db

10_sp_24_e
9_sp_24_p
8_sp_23_e
7_sp_23_p
6_sp_22_e
5_sp_22_p
4_sp_21_p
3_sp_20_e
2_sp_19_p
1_sp_18_p


,10_sp_24_e,9_sp_24_p,8_sp_23_e,7_sp_23_p,6_sp_22_e,5_sp_22_p,4_sp_21_p,3_sp_20_e,2_sp_19_p,1_sp_18_p
sc2736,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rl694,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
yz2655,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ac2558,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mjc456,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
mfd1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
aaj43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
xy253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
cp499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [187]:
db_groupby = db.groupby(db.index).sum()
db_groupby.index.name = "id"
display(db_groupby)
db_groupby.to_csv("db.csv")

,10_sp_24_e,9_sp_24_p,8_sp_23_e,7_sp_23_p,6_sp_22_e,5_sp_22_p,4_sp_21_p,3_sp_20_e,2_sp_19_p,1_sp_18_p
id,,,,,,,,,,
17hfeldman,0,0,0,0,0,0,0,1,0,0
aa2224,0,0,0,0,0,0,0,1,0,0
aa2226,0,0,0,0,0,0,0,1,0,0
aa2229,0,0,0,0,0,0,1,0,0,0
aa2247,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
zz547,0,1,0,0,0,0,0,0,0,0
zz768,0,1,0,0,0,0,0,0,0,0
zza2,0,2,0,1,0,0,0,0,0,0


Old Stuff

In [ ]:
all = pd.read_csv("all.csv")
all["First Name"] = all["Recipient's First Name"] = all["Recipient\'s Full Name"].str.split(n=1, expand=True)[0]
all

In [ ]:
all_groupby = all.groupby(["Recipient NetID", "Recipient's First Name"])["Recipient NetID"].count().to_frame(name="Num Messages").reset_index()
all_groupby["Email"] = all_groupby["Recipient NetID"] + "@cornell.edu"
all_groupby["Access Code"] = all_groupby.index
all_groupby.to_csv("all_groupby.csv")
all_groupby

In [ ]:
# open the pdf file
reader = pypdf.PdfReader("all.pdf")

# get number of pages
num_pages = len(reader.pages)

# extract text and do the search
for page in reader.pages:
    text = page.extract_text() 
    for netID, accessCode in zip(all_groupby["Recipient NetID"], all_groupby["Access Code"]):
        writer = pypdf.PdfWriter()
        res_search = re.search(netID, text)
        if res_search is not None:
            # print(res_search)

            file_name = netID + "_" + str(accessCode) + ".pdf"

            if os.path.exists("pdfs/" + file_name):
                writer.append("pdfs/" + file_name)

            writer.add_page(page)
            with open("pdfs/" + file_name, 'wb') as out:
                writer.write(out)

Create MailMerge

In [ ]:
post = pd.read_csv("post_lifted_access.csv")
post

In [ ]:
post_merge_list = all_groupby.loc[all_groupby["Recipient NetID"].isin(post["NetID"])]
post_merge_list.to_csv("post_merge_list.csv")
post_merge_list

In [ ]:
# Sanity check (we want this to be empty).  If it's not, either the NetID was not actually lifted or there's a space in the post csv or something
post.loc[~post["NetID"].isin(post_merge_list["Recipient NetID"])]